## PDF Query Langchain Project 

### Dependencies 



In [1]:
!pip install -q cassio datasets langchain openai tiktoken 



In [2]:
from langchain.vectorstores.cassandra import Cassandra #this is db 
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset 

import os
import cassio


/Users/sanginkang/Desktop/LangChain/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
! pip install PyPDF2

In [4]:
from PyPDF2 import PdfReader


In [5]:
ASTRADB_APPLICATION_TOKEN = "AstraCS:xDinaAcaIFQJHjMziuQNopvI:1ccebab198afc3b69b822590fa7b5a4247b9000553f5d1d5bd0746503c8f139d"
ASTRA_DB_ID = "5e83bc72-e3bf-48c8-a325-5b034181dabe"
OPENAPI_KEY = os.getenv("OPEN_API_KEY")

### Read Document

In [6]:
pdfreader = PdfReader('World Bank Annual Report 2024.pdf')

In [7]:
from typing_extensions import Concatenate

raw_text = ' '
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content 

### Initialize DB 

In [8]:
cassio.init(token=ASTRADB_APPLICATION_TOKEN, database_id= ASTRA_DB_ID)


In [9]:
llm = OpenAI(api_key = OPENAPI_KEY )
embedding = OpenAIEmbeddings(api_key=OPENAPI_KEY)

/var/folders/5q/9x8zl08n4xz0vrx4cygd2lyw0000gn/T/ipykernel_75371/2108375935.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(api_key = OPENAPI_KEY )
/var/folders/5q/9x8zl08n4xz0vrx4cygd2lyw0000gn/T/ipykernel_75371/2108375935.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(api_key=OPENAPI_KEY)


# Create Vector Store 


In [10]:
astra_vector_store = Cassandra(
    embedding = embedding,
    table_name = "QA_Demo",
    session = None ,
    keyspace = None

)

## Create chunks of text and placei n store

In [11]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len

)

text = text_splitter.split_text(raw_text)





In [12]:
text[:50]

['A BETTER \nBANK FOR \nA BETTER \nWORLD\nANNUAL REPORT 20241\nTHE WORLD BANK ANNUAL REPORT 2024 THE WORLD BANK ANNUAL REPORT 2024CONTENTS\nThis annual report, which covers the period from \nJuly 1, 2023, to June 30, 2024, has been prepared \nby the Executive Directors of both the International \nBank for Reconstruction and Development (IBRD) and \nthe International Development Association (IDA)—\ncollectively known as the World Bank—in accordance \nwith the respective bylaws of the two institutions. \nAjay Banga, President of the World Bank Group and \nChairman of the Board of Executive Directors, has \nsubmitted this report, together with the accompanying \nadministrative budgets and audited financial \nstatements, to the Board of Governors. \nAnnual reports for the other World Bank Group',
 'submitted this report, together with the accompanying \nadministrative budgets and audited financial \nstatements, to the Board of Governors. \nAnnual reports for the other World Bank Group \nin

In [13]:
astra_vector_store.add_texts(text[:50])

print("Inserted %i headlines." % len(text[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore = astra_vector_store)

Inserted 50 headlines.


In [15]:
first_q = True 
while True: 
    if first_q:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhats your next questions (or type 'quit' to exit): ").strip()
    if query_text.lower() == 'quit':
        break
    if query_text == "":
        continue 
    first_q = False 
    print("\nQuestion: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("\nAnswer: \"%s\"\n" %  answer)

    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k = 4):
        print("     [%0.4f] \"%s....\"" % (score, doc.page_content[:84]))



Question: "What was the main contribution to Tanzania?"

Answer: "The results-based financing education program that led to 1.6 million students enrolling in preprimary and primary schools."

     [0.8946] "region, including $2.9 billion in IBRD commitments and $12.7 billion in IDA 
commitm...."
     [0.8946] "region, including $2.9 billion in IBRD commitments and $12.7 billion in IDA 
commitm...."
     [0.8946] "region, including $2.9 billion in IBRD commitments and $12.7 billion in IDA 
commitm...."
     [0.8945] "region, including $2.9 billion in IBRD commitments and $12.7 billion in IDA 
commitm...."

Question: "What are some of the major contribution that happened in this report"

Answer: "This is not mentioned in the context given."

     [0.9006] "submitted this report, together with the accompanying 
administrative budgets and au...."
     [0.9006] "submitted this report, together with the accompanying 
administrative budgets and au...."
     [0.9006] "submitted this report, t